In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse,constructBasicDict

### Hyper parameters setting

In [2]:
hparams = {
    'npratio':4,
    'mode':'train',
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cuda:1',
    'attrs': ['title'],
    'news_id':True,
    'k': 0,
    'validate':False
}

device = torch.device(hparams['device'])
# torch.cuda.set_device(device)

### View data

In [3]:
vocab, loaders = prepare(hparams)

In [4]:
# loader_train
a = next(iter(loaders[0]))
# loader_dev
b = next(iter(loaders[1]))

In [5]:
a,b

({'user_index': tensor([[ 509],
          [  81],
          [ 833],
          [1614],
          [1614],
          [1056],
          [ 409],
          [ 409],
          [ 409],
          [ 566]]),
  'his_mask': tensor([[[False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [False],
           [

### Construct necessary dictionaries
- already done and the results are in `data/dictionaries`

In [ ]:
# constructBasicDict('demo',attrs=['title'],path='/home/peitian_zhang/Data/MIND')
constructBasicDict('small',attrs=['title'],path='/home/peitian_zhang/Data/MIND')
constructBasicDict('large',attrs=['title'],path='/home/peitian_zhang/Data/MIND')

### Tailor Data to demo size

In [ ]:
# tailor 2000 impressions from MINDsmall_train to form MINDdemo_trai
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)

tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [ ]:
analyse(hparams)